## Notebook for the experiment of building **DeCaf** (**De**sign **C**l**a**ssi**f**ier)

### Objective
The main objective is to classify discussions from pull request, issue tracker commit messages and code comments as `design` or `general`. We also want to make the classifier cross project compatible.

### Data Collection
- We intent to have three types of data. One data is to train the `Word Embedding` model. As `Word Embedding` requires structured form of literature, we have used sentences from literatures(ex. papers and books). Also we have restricted our choice of papers and books to only from the Software Engineering domain for keep the context of our `Word Embedding` model restricted to Software Engineering. Our `Word Embedding` model will be used to vectorize our train data.
- We have collected our train data from Stack Overflow questions, answers and comments. We have collected data and classified them as `design` or `general` based on the tag. For example, questions and answers that contain `design-patterns` or `software-design` falls under the class of `design` while data tagged as `javascript` or `django` as classified as `general`

### Data Cleaning
Raw data can have a lot of noise. Specially when scraped from documents of website, it can contain a lot of misinformation in the form of names, punctuations, numbers(ex. years), misspelled and incompleted words. Also it can have a lot of stopwords that can make the model confused. We have removed all this to make our data as clean as possible. After the cleaning process, out data only contains words that are not stopwords, present in the english dictionary and has lenght greater than three.

In [0]:
from google.colab import drive
drive.mount("/content/drive")

### Assign Data location
- literature: holds the data for word embedding

In [0]:
literature_file = "/content/drive/My Drive/UVic-RA/DeCaf/data/literature.txt"
we_model_file = "/content/drive/My Drive/UVic-RA/DeCaf/model/we.bin"

### Create Word Embedding
- Use fasttext for word embedding
1. we take literature.txt as our input data.
2. we train the classifier upsupervised since we just want to group the data according to the similarity.
3. we have used skipgram as we have observe that skipgram models works better with subword information that cbow
4. we are taking words with character number from 4-20. since we are removing every word less than three characters, its not important to take the characters less than 4 characters. Also, the design words seems to be on the bigger side. for ex. reproduceability contains 16 characters. we are considering characters upto 25 characters.
5. We are taking 300 dimension of the words. Also looping for 10 epochs. Both because the training corpus is relatively small.

In [0]:
!pip install fasttext

### Train the `Word Embedding` model and save it as we.bin
- Train and save the model if the model is not present
- Load the model from disk if preselt

In [0]:
import os
from datetime import datetime as dt
import fasttext as ft

if not os.path.exists(we_model_file):
  prints(str(dt.now())+' Training Word Embedding model...')
  model = ft.train_unsupervised(literature_file, "skipgram", minn=4, maxn=25, dim=300, epoch=10)
  model.save_model(we_model_file)
  print(str(dt.now())+' Model trained and saved successfully')
else:
  print(str(dt.now())+' Loading Word Embedding model from disk...')
  model = ft.load_model(we_model_file)
  print(str(dt.now())+' Model loaded successfully.')

2020-03-16 08:59:03.262021 Loading Word Embedding model from disk...


2020-03-16 08:59:12.913266 Model loaded successfully.


### Playing around with the model

Get the dimention of the model

In [0]:
model.get_dimension()

300

Get the words of the model... For a demo... 

In [0]:
model.get_words()

Get the vector of a word

In [0]:
model.get_word_vector('Maintainability')

Get the vector of a sentence

In [0]:
model.get_sentence_vector("Add performance tracker to active admin jobs")

### Read train data
We use three types of data from training the model. We scraped question, answer and comment data from Stack Overflow and tagged them `design` or `general`. The tagging was done automatically based on the original tag of the question. Then we process our data as we did before for our word embedding data to clear noise. We also did some additional processing to our train data. We only took those documents that contains more than 10 words. For the others, we discarded them. After processing, we got 1,00,000 documents(50,000-design, 50,000-general) for `questions.csv`, 40,000 documents(20,000-design, 20,000-general) for `answers.csv` and 60,000 documents(30,000-design, 30,000-general) for `comments.csv`.

Our train data is complete noise free, stopwords free and long documents of more than 10 words per document. All the data is randomly distributed.

#### Assign train data location


In [0]:
train_question_file = "/content/drive/My Drive/UVic-RA/DeCaf/data/train_data/questions.csv"
train_answer_file = "/content/drive/My Drive/UVic-RA/DeCaf/data/train_data/answers.csv"
train_comment_file = "/content/drive/My Drive/UVic-RA/DeCaf/data/train_data/comments.csv"

#### Read the data with pandas

In [0]:
import pandas as pd
from prettytable import PrettyTable

train_question = pd.read_csv(train_question_file)
train_answer = pd.read_csv(train_answer_file)
train_comment = pd.read_csv(train_comment_file)

#### Explore the train data

In [21]:
print("Question train data")
print(train_question.head())
print("Answer train data")
print(train_answer.head())
print("Comment train data")
print(train_comment.head())

Question train data
                                            question    label
0  android acknowledge status when android acknow...  general
1  error when error when check error name uncaugh...  general
2  quickly retrieve property info when property k...   design
3  define good closed taking class called first l...   design
4  rails create service class factory trying writ...  general
Answer train data
                                              answer    label
0  return modified object instead attempting modi...   design
1  think chaining popular populate request data p...   design
2  good start kind design pattern inside grade gr...   design
3  note referencing aware rules apply frameworks ...   design
4  need method entity interface entity want entit...  general
Comment train data
                                             comment    label
0  launch experience instruments pretty completel...  general
1  dangerous path walk down instead letting insta...   design
2  point writ

In [20]:
table = PrettyTable()

table.field_names = ["Dataset Name", "Shape", "# of design data", "# of general data"]

qd = train_question[train_question['label']=='design']
qg = train_question[train_question['label']=='general']

ad = train_answer[train_answer['label']=='design']
ag = train_answer[train_answer['label']=='general']

cd = train_comment[train_comment['label']=='design']
cg = train_comment[train_comment['label']=='general']

table.add_row(["Question", train_question.shape, qd.shape[0], qg.shape[0]])
table.add_row(["Answer", train_answer.shape, ad.shape[0], ag.shape[0]])
table.add_row(["Comment", train_comment.shape, cd.shape[0], cg.shape[0]])


print(table)

+--------------+-------------+------------------+-------------------+
| Dataset Name |    Shape    | # of design data | # of general data |
+--------------+-------------+------------------+-------------------+
|   Question   | (100000, 2) |      50000       |       50000       |
|    Answer    |  (40000, 2) |      20000       |       20000       |
|   Comment    |  (60000, 2) |      30000       |       30000       |
+--------------+-------------+------------------+-------------------+
